# Establishing a database connection


In [2]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote

server_name   = "localhost"
database_name = "BookStore"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};Trusted_Connection=yes"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)


Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=BookStore;Trusted_Connection=yes 

Successfully connected to BookStore!


### To protect against SQL injection, use parameterized queries instead of string concatenation

In [30]:
from sqlalchemy import create_engine, text


# Define a function to search for books based on a search term
def search_books(title_query):
    
    # Use parameterization to protect against SQL injection
    query = text("""
        SELECT  B.title, I.Quantity as num_copies, S.StoreName
        FROM Books B
        join InventoryBalance I on B.ISBN13= I.ISBN13
		join Stores S on I.StoreID = S.ID
        WHERE b.title ILIKE :title_query
        GROUP BY B.title, I.Quantity, S.StoreName
    """)
    # Use ILIKE instead of LIKE to make the search case-insensitive

    with engine.connect() as conn:
        result = conn.execute(query, {'title_query': f'%{title_query}%'})
        books = result.fetchall()
        return books

#  enter a search term and display the results
title_query = input("Enter the title of the book that looking for: ")
books = search_books(title_query)
if not books:
    print("No matches found")
else:
    for book in books:
        print(f"{book.title} is available at {book.store_name} with {book.num_copies} copies.")




ProgrammingError: (pyodbc.ProgrammingError) ('42000', "[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]An expression of non-boolean type specified in a context where a condition is expected, near 'ILIKE'. (4145) (SQLExecDirectW); [42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]Statement(s) could not be prepared. (8180)")
[SQL: 
        SELECT  B.title, I.Quantity as num_copies, S.StoreName
        FROM Books B
        join InventoryBalance I on B.ISBN13= I.ISBN13
		join Stores S on I.StoreID = S.ID
        WHERE b.title ILIKE ?
        GROUP BY B.title, I.Quantity, S.StoreName
    ]
[parameters: ('%The Mist%',)]
(Background on this error at: https://sqlalche.me/e/20/f405)